<a href="https://colab.research.google.com/github/DHasanC/MLHW1/blob/master/MachineLearning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import cifar10
from keras import Sequential
from keras.utils import np_utils
from keras.optimizers import Adagrad, Adam
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Activation, Flatten, BatchNormalization
from keras import regularizers
import os
import numpy as np

In [0]:
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

indices = np.random.permutation(len(train_images))

val_indices = indices[0:int(len(indices)*0.1)] 
train_indices = indices[int(len(indices)*0.1):]

val_images, val_labels = train_images[val_indices], train_labels[val_indices]

Train_images, Train_labels = train_images[train_indices], train_labels[train_indices]

X_train = (Train_images/255)
y_train = np_utils.to_categorical(Train_labels, 10)

X_val = (val_images/255)
y_val = np_utils.to_categorical(val_labels, 10)

X_test = (test_images/255)
y_test = np_utils.to_categorical(test_labels, 10)

we define model without dropout layers and data augmentation

In [0]:
model1 = Sequential()
model1.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))

model1.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
 
model1.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model1.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model1.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))

model1.add(Flatten()) 
model1.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model1.add(Dense(64, activation= 'relu'))
model1.add(Dense(10, activation='softmax'))

model1.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 32, 48)        13872     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 16, 16, 128)       55424     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 4, 4, 256)         295168    
_________________________________________________________________
flatten_5 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               524416    
__________

In [0]:
# Initiate Adagrad/RMSprop optimizer
model1.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

## Train the model and get the training and validation loss
history = model1.fit(X_train, y_train, batch_size= 40, epochs= 120,
                    verbose=1, validation_data=(X_val, y_val))

Train on 45000 samples, validate on 5000 samples
Epoch 1/120
45000/45000 [==============================] - 22s 488us/step - loss: 1.5478 - acc: 0.4717 - val_loss: 1.1637 - val_acc: 0.6194
Epoch 2/120
45000/45000 [==============================] - 21s 470us/step - loss: 1.0207 - acc: 0.6638 - val_loss: 1.0028 - val_acc: 0.6738
Epoch 3/120
45000/45000 [==============================] - 21s 475us/step - loss: 0.8487 - acc: 0.7284 - val_loss: 0.9289 - val_acc: 0.7084
Epoch 4/120
45000/45000 [==============================] - 21s 469us/step - loss: 0.7394 - acc: 0.7664 - val_loss: 0.9182 - val_acc: 0.7106
Epoch 5/120
45000/45000 [==============================] - 21s 470us/step - loss: 0.6540 - acc: 0.7987 - val_loss: 0.8819 - val_acc: 0.7274
Epoch 6/120
45000/45000 [==============================] - 21s 471us/step - loss: 0.5803 - acc: 0.8256 - val_loss: 0.8713 - val_acc: 0.7386
Epoch 7/120
45000/45000 [==============================] - 21s 471us/step - loss: 0.5157 - acc: 0.8504 - val_lo

In [0]:
scores = model1.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 175us/step
Test loss: 2.013573998451233
Test accuracy: 0.726


model2 (includes dropout layers)

In [0]:
model2 = Sequential()
model2.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.1))


model2.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
model1.add(Dropout(0.5))
 
model2.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.1))

model2.add(Flatten())
model2.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model2.add(Dense(64, activation= 'relu'))
model2.add(Dense(10, activation='softmax'))

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 48)        13872     
_________________________________________________________________
dropout_10 (Dropout)         (None, 32, 32, 48)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 32, 32, 64)        27712     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 8, 8, 256)         147712    
__________

In [0]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

history = model2.fit(X_train, y_train, batch_size= 40, epochs= 30,
                    verbose=1, validation_data=(X_val, y_val))

Train on 45000 samples, validate on 5000 samples
Epoch 1/30
45000/45000 [==============================] - 33s 723us/step - loss: 1.6400 - acc: 0.4502 - val_loss: 1.2711 - val_acc: 0.5920
Epoch 2/30
45000/45000 [==============================] - 31s 687us/step - loss: 1.1179 - acc: 0.6340 - val_loss: 1.1479 - val_acc: 0.6366
Epoch 3/30
45000/45000 [==============================] - 31s 691us/step - loss: 0.9297 - acc: 0.7035 - val_loss: 1.0330 - val_acc: 0.6706
Epoch 4/30
45000/45000 [==============================] - 31s 685us/step - loss: 0.8038 - acc: 0.7520 - val_loss: 0.9651 - val_acc: 0.6972
Epoch 5/30
45000/45000 [==============================] - 31s 683us/step - loss: 0.6981 - acc: 0.7912 - val_loss: 0.9603 - val_acc: 0.7038
Epoch 6/30
45000/45000 [==============================] - 31s 684us/step - loss: 0.6057 - acc: 0.8231 - val_loss: 0.9747 - val_acc: 0.7092
Epoch 7/30
45000/45000 [==============================] - 31s 683us/step - loss: 0.5270 - acc: 0.8542 - val_loss: 1.0

In [0]:
## Score trained model.
scores = model2.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 195us/step
Test loss: 1.696219270992279
Test accuracy: 0.7332


model 3 with data augmentation

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=60,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        fill_mode='nearest')

In [0]:
model3 = Sequential()
model3.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))


model3.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))

model3.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model3.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))

model3.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))

model3.add(Flatten())
model3.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model3.add(Dense(64, activation= 'relu'))
model3.add(Dense(10, activation='softmax'))

In [0]:
model3.compile(loss='categorical_crossentropy', 
               optimizer='adagrad', metrics=['accuracy'])
 
model3.fit_generator(datagen.flow(X_train, y_train, batch_size=40),
                    steps_per_epoch=train_images.shape[0]/ 40,epochs= 100,
                    verbose=1,validation_data=(X_val,y_val))

Epoch 1/100
1250/1250 [==============================] - 45s 36ms/step - loss: 2.0105 - acc: 0.3338 - val_loss: 1.4950 - val_acc: 0.4842
Epoch 2/100
1250/1250 [==============================] - 44s 35ms/step - loss: 1.5777 - acc: 0.4522 - val_loss: 1.3677 - val_acc: 0.5322
Epoch 3/100
1250/1250 [==============================] - 43s 35ms/step - loss: 1.4499 - acc: 0.5008 - val_loss: 1.3049 - val_acc: 0.5622
Epoch 4/100
1250/1250 [==============================] - 43s 34ms/step - loss: 1.3587 - acc: 0.5380 - val_loss: 1.2006 - val_acc: 0.5976
Epoch 5/100
1250/1250 [==============================] - 43s 34ms/step - loss: 1.2910 - acc: 0.5647 - val_loss: 1.1603 - val_acc: 0.6188
Epoch 6/100
1250/1250 [==============================] - 45s 36ms/step - loss: 1.2559 - acc: 0.5763 - val_loss: 1.1849 - val_acc: 0.6090
Epoch 7/100
1250/1250 [==============================] - 45s 36ms/step - loss: 1.2109 - acc: 0.5950 - val_loss: 1.1297 - val_acc: 0.6264
Epoch 8/100
1250/1250 [==================

In [0]:
scores = model3.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 196us/step
Test loss: 0.8596715850830078
Test accuracy: 0.7439


model 4: includes dropout and data augmentation

In [0]:
model4 = Sequential()
model4.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(Dropout(0.1))

model4.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(Dropout(0.25))

model4.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
model4.add(Dropout(0.5))
 
model4.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(Dropout(0.1))

model4.add(Flatten())
model4.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model4.add(Dense(64, activation= 'relu'))
model4.add(Dense(10, activation='softmax'))

In [0]:
model4.compile(loss='categorical_crossentropy', 
               optimizer='adagrad', metrics=['accuracy'])

model4.fit_generator(datagen.flow(X_train, y_train, batch_size=40),
                    steps_per_epoch=train_images.shape[0]/ 40,epochs= 100,
                    verbose=1,validation_data=(X_val,y_val))

Epoch 1/100
1250/1250 [==============================] - 47s 38ms/step - loss: 2.1812 - acc: 0.3125 - val_loss: 1.7274 - val_acc: 0.4052
Epoch 2/100
1250/1250 [==============================] - 45s 36ms/step - loss: 1.7466 - acc: 0.3928 - val_loss: 1.6017 - val_acc: 0.4492
Epoch 3/100
1250/1250 [==============================] - 45s 36ms/step - loss: 1.6426 - acc: 0.4297 - val_loss: 1.4703 - val_acc: 0.5034
Epoch 4/100
1250/1250 [==============================] - 45s 36ms/step - loss: 1.5710 - acc: 0.4593 - val_loss: 1.4011 - val_acc: 0.5212
Epoch 5/100
1250/1250 [==============================] - 45s 36ms/step - loss: 1.5151 - acc: 0.4799 - val_loss: 1.3541 - val_acc: 0.5438
Epoch 6/100
1250/1250 [==============================] - 45s 36ms/step - loss: 1.4770 - acc: 0.4959 - val_loss: 1.3448 - val_acc: 0.5492
Epoch 7/100
1250/1250 [==============================] - 46s 36ms/step - loss: 1.4417 - acc: 0.5051 - val_loss: 1.3503 - val_acc: 0.5460
Epoch 8/100
1250/1250 [==================

k-fold validation

In [0]:
from sklearn.model_selection import KFold

X_train = train_images.reshape(train_images.shape[0], 32, 32, 3)
y_train_Kfold = np_utils.to_categorical(train_labels, 10)
X_train_Kfold = (X_train/255).astype('float32')


scores = []
cv = KFold(n_splits=5, random_state=42, shuffle=False)
for train_index, test_index in cv.split(X_train_Kfold):
  
    X_train, X_val, y_train, y_val = X_train_Kfold[train_index], X_train_Kfold[test_index], y_train_Kfold[train_index], y_train_Kfold[test_index]

    
    model4.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    model4.fit(X_train, y_train)

   
    scores.append(model4.evaluate(X_test, y_test))
    
print("K-fold cross validation DONE...")
print("The Cross validation Score [Accuracy]",np.mean(scores[1]))

Epoch 1/1
10000/10000 [==============================] - 2s 219us/step
Epoch 1/1
10000/10000 [==============================] - 2s 206us/step
Epoch 1/1
10000/10000 [==============================] - 2s 208us/step
Epoch 1/1
10000/10000 [==============================] - 2s 211us/step
Epoch 1/1
10000/10000 [==============================] - 2s 213us/step
K-fold cross validation DONE...
The Cross validation Score [Accuracy] 0.8365297165870667
